This script converts all .bk2 Files to training videos (.mp4) using gym-retro.

In [ ]:
import os
import subprocess
import shutil

# Create and install dependencies in a virtual environment
subprocess.run(['virtualenv', '-p', 'python3.8', 'gym_venv'])
subprocess.run(['gym_venv/bin/pip', 'install', 'gym==0.21.0', 'gym-retro==0.8.0', 'ffmpeg'])

# Import Shinobi III ROM
subprocess.run(['gym_venv/bin/python3', '-m', 'retro.import', '../rom'])

# Define base paths
base_path = '../data/raw_data/sub-01'
video_base_path = '../output/videos/videos_full'

# Ensure the output directory exists
os.makedirs(video_base_path, exist_ok=True)

start_session = 3  # ses-003
end_session = 14  # ses-013
venv_python_path = 'gym_venv/bin/python3'

# Loop through session directories
for session in range(start_session, end_session + 1):
    session_str = f'ses-{session:03d}'
    session_path = os.path.join(base_path, session_str)
    
    if not os.path.exists(session_path):
        print(f"Session directory {session_path} does not exist. Skipping.")
        continue
    
    for file in os.listdir(session_path):
        if file.endswith('.bk2'):
            input_file = os.path.join(session_path, file)
            output_bk2_file = os.path.join(video_base_path, file)
            output_mp4_file = os.path.join(video_base_path, file.replace('.bk2', '.mp4'))

            if os.path.exists(output_mp4_file):
                print(f"{output_mp4_file} already exists. Skipping.")
                continue
            
            shutil.copy(input_file, output_bk2_file)
            print(f"Copied {input_file} to {output_bk2_file}")
            
            command = [venv_python_path, '-m', 'retro.scripts.playback_movie', output_bk2_file, '--lossless', 'mp4']
            subprocess.run(command)

            if os.path.exists(output_mp4_file):
                os.remove(output_bk2_file)
                print(f"Removed {output_bk2_file}")
            else:
                print(f"Expected output file {output_mp4_file} not found. Please check the playback script output.")

# Remove any remaining .bk2 files in the output folder
for file in os.listdir(video_base_path):
    if file.endswith('.bk2'):
        bk2_file_path = os.path.join(video_base_path, file)
        os.remove(bk2_file_path)
        print(f"Removed remaining .bk2 file: {bk2_file_path}")